In [25]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import math
import random
import imutils

Get templates

In [2]:
def get_templates():
    templates = {}
    for x in os.listdir("small_templates"):
        templates[x] = cv2.imread("small_templates/" + x)
    
    template_labels = []   
    c=0
    for t in templates:
        templates[t] = cv2.cvtColor(templates[t], cv2.COLOR_BGR2GRAY) # convert to grey
        templates[t] = cv2.medianBlur(templates[t],5) # blur
        templates[t] = cv2.Canny(templates[t], 50, 200, apertureSize = 3) # edge detection
        
        template_labels.append((c,templates[t]))
        c+=1
    return template_labels
        
        
#         cv2.imshow("labels", template_labels[c][1])
#         cv2.waitKey(0)

#         print(template_labels[c][0])

#         cv2.imshow("templates", templates[t])
#         cv2.waitKey(0)
        

In [3]:
template_labels = get_templates()

Get images

In [4]:
def get_images():
    cassiopeia = {}
    hercules = {}
    lyra = {}
    for x in os.listdir("raw/cassiopeia"):
        cassiopeia[x] = cv2.imread("raw/cassiopeia/" + x)
    for x in os.listdir("raw/hercules"):
        hercules[x] = cv2.imread("raw/hercules/" + x)
    for x in os.listdir("raw/lyra"):
        lyra[x] = cv2.imread("raw/lyra/" + x)

    image_labels = []   

    for i in cassiopeia:        
        image_labels.append((0,cassiopeia[i]))
    for i in hercules:        
        image_labels.append((1,hercules[i]))
    for i in lyra:        
        image_labels.append((2,lyra[i]))
#         cv2.imshow("images", images[i])
#         cv2.waitKey(0)
        
    return image_labels

In [5]:
image_labels = get_images()

Skew images

In [6]:
def skew_images(raw):
#     filtered = [item[1] for item in raw]
#     filtered_labels = [item[0] for item in raw]
#     print(filtered_labels)
    data = []
    for photo in raw:
        for x in range(5):
            new = photo[1].copy()
            row, col, _ = new.shape

            kvalue = random.randint(1,6)
            ksize = (kvalue,kvalue)
            blurred = cv2.blur(new,ksize)

            number_of_pixels = random.randint(300, 10000)
            for i in range(number_of_pixels):

                y_coord=random.randint(0, row - 1)

                x_coord=random.randint(0, col - 1)

                blurred[y_coord][x_coord] = 255

            number_of_pixels = random.randint(300 , 10000)
            for i in range(number_of_pixels):

                y_coord=random.randint(0, row - 1)

                x_coord=random.randint(0, col - 1)

                blurred[y_coord][x_coord] = 0
                
            data.append((photo[0],blurred))
            
        
        
#     for photo in data:
#         cv2.imshow("skewed", photo[1])
#         cv2.waitKey(0)
        
    return data
        
    



# median1 = cv2.medianBlur(filtered,5)

# # edge detection
# edges1 = cv2.Canny(median, 50, 200, apertureSize = 3)



# cv2.imshow("Template", blurred)
# cv2.waitKey(0)

In [7]:
data = skew_images(image_labels)

In [8]:
def filter_images(data):
    dataset = []
    for photo in data:
        temp = cv2.medianBlur(photo[1],5) # blur
        temp = cv2.Canny(temp, 50, 200, apertureSize = 3) # edge detection
        dataset.append((photo[0],temp))
        
#     for photo in dataset:
#         cv2.imshow("test", photo[1])
#         cv2.waitKey(0)

        
    return dataset
    

In [9]:
dataset = filter_images(data)

In [33]:
def match(photo, template):
    img = photo[1]
    test = img.copy()
    img_val = photo[0]
    temp = template[1]
    temp_val = template[0]
    
    result = cv2.matchTemplate(img, temp, cv2.TM_CCOEFF_NORMED)
    (minVal, maxVal, minLoc, maxLoc) = cv2.minMaxLoc(result)

    (startX, startY) = maxLoc
    endX = startX + temp.shape[1]
    endY = startY + temp.shape[0]

    

    threshold = 0.5
    flag = False
    if np.amax(result) > threshold:
        flag = True
        cv2.rectangle(test, (startX, startY), (endX, endY), (255, 0, 0), 3)
        cv2.imshow("test", test)
        cv2.waitKey(0)
        
#     print(flag)
#     print("Img Val: "+str(img_val))
#     print("Temp Val: "+str(temp_val))
    
    return flag, img_val, temp_val

#     cv2.imshow("Output", new)
#     cv2.waitKey(0)

#     return flag

In [34]:
correct = 0
should_be_true = 0
wrong = 0
total = 0
for photo in dataset:
    for t in template_labels:
        flag, img_val, temp_val = match(photo,t)
        if flag == True and img_val == temp_val:
            correct+=1
        elif flag == False and img_val == temp_val:
            should_be_true+=1
        elif flag == True and img_val != temp_val:
            wrong+=1
        total+=1
        
print("Correct: "+str(correct))
print("Should be correct: "+str(should_be_true))
print("Wrong: "+str(wrong))
print("Total: "+str(total))

Correct: 8
Should be correct: 17
Wrong: 0
Total: 75


In [ ]:
# def test(img, temp):
#     image = img[1]
#     template = temp[1]
#     found = None
#     best = None
#     (tH, tW) = template.shape[:2]
#     # loop over the scales of the image
#     for scale in np.linspace(0.2, 1.0, 20)[::-1]:
#         # resize the image according to the scale, and keep track
#         # of the ratio of the resizing
#         resized = imutils.resize(image, width = int(image.shape[1] * scale))
#         r = image.shape[1] / float(resized.shape[1])
#         # if the resized image is smaller than the template, then break
#         # from the loop
#         if resized.shape[0] < tH or resized.shape[1] < tW:
#             break
#         c=0
#         for x in range(360):
#             image_center = tuple(np.array(resized.shape[1::-1]) / 2)
#             rot_mat = cv2.getRotationMatrix2D(image_center, c, 1.0)
#             rotated = cv2.warpAffine(resized, rot_mat, resized.shape[1::-1], flags=cv2.INTER_LINEAR)
            
#             result = cv2.matchTemplate(rotated, template, cv2.TM_CCOEFF)
#             (_, maxVal, _, maxLoc) = cv2.minMaxLoc(result)

#             if found is None or maxVal > found[0]:
#                 best = rotated
#                 found = (maxVal, maxLoc, r)
#     (_, maxLoc, r) = found
#     (startX, startY) = (int(maxLoc[0] * r), int(maxLoc[1] * r))
#     (endX, endY) = (int((maxLoc[0] + tW) * r), int((maxLoc[1] + tH) * r))
    
#     cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
#     cv2.imshow("Image", image)
#     cv2.waitKey(0)

In [ ]:
# test(dataset[0], template_labels[0])